In [1]:
import numpy as np
import pandas as pd
import contractions
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vadla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
reviews = pd.read_csv('Reviews.csv')
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
# Check if there are any missing (NaN) values in each column.
reviews.isna().any()

Id                        False
ProductId                 False
UserId                    False
ProfileName                True
HelpfulnessNumerator      False
HelpfulnessDenominator    False
Score                     False
Time                      False
Summary                    True
Text                      False
dtype: bool

In [4]:
# Extracting only Text and Score Columns and Dropping the NAN values
reviews = reviews[['Text','Score']].copy()
reviews.dropna(inplace=True)

In [5]:
#if score> 3, set score = 1
#if score<=2, set score = 0
#if score == 3, remove the rows.
reviews.loc[reviews['Score']<=2,'Score'] = 0
reviews.loc[reviews['Score']>3,'Score']  = 1
reviews.drop(reviews[reviews['Score']==3].index,inplace=True) 

In [6]:
# Create a set of English stopwords using the 'stopwords' module
stop_words = set(stopwords.words('english'))
def remove_tags(sentence):
    # Remove HTML tags using a regular expression
    sentence = re.sub('<.*?>','',sentence)
    # Expand contractions (e.g., "isn't" -> "is not")
    sentence = contractions.fix(sentence)
    # Keep only alphabetic words (removes punctuation and numbers)
    sentence = " ".join(re.findall(r'\b[a-zA-Z]+\b', sentence))
    # Convert the sentence to lowercase to ensure uniformity
    sentence = sentence.lower()
    # Remove stopwords from the sentence
    sentence = " ".join([word for word in sentence.split() if word.lower() not in stop_words])
    # Return the cleaned sentence
    return sentence

In [7]:
# Enable progress bars for pandas operations using tqdm
tqdm.pandas()
# Apply the 'remove_tags' function to the 'Text' column of the 'reviews' dataframe
# The progress_apply method displays a progress bar during the operation
reviews['Text'] = reviews['Text'].progress_apply(lambda cw : remove_tags(cw))

100%|███████████████████████████████████████████████████████████████████████| 525814/525814 [00:26<00:00, 20079.53it/s]


In [8]:
# Displaying the first 5 rows of the 
reviews.head(5)

,Text,Score
0,bought several vitality canned dog food produc...,1
1,product arrived labeled jumbo salted peanuts p...,0
2,confection around centuries light pillowy citr...,1
3,looking secret ingredient robitussin believe f...,0
4,great taffy great price wide assortment yummy ...,1


In [9]:
# Save the 'reviews' DataFrame to a CSV file named 'preprocessed.csv',
# without including the index column in the output file.
reviews.to_csv('preprocessed.csv', index=False)